<a href="https://colab.research.google.com/github/MoSkibidi/The_Art_Of_Computer_System/blob/main/assembler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# แอสเซมเบอร์ (Assembler)

*ดัดแปลงจากโน้ตบุ๊คต้นฉบับโดย ผศ.ดร.จิตร์ทัศน์ ฝักเจริญผล ภาคต้น ปีการศึกษา 2563 และอ้างอิงเนื้อหาจากตำรา [The Elements of Computing Systems โดย Nisan และ Schocken](https://www.nand2tetris.org/book)*

**เอกสารประกอบ:** นิสิตสามารถอ่านเอกสารประกอบได้ที่ [รากฐานระบบคอมพิวเตอร์: จากเม็ดทรายสู่ Hello, World! บทที่ 6 แอสเซมเบลอร์](https://ecourse.cpe.ku.ac.th/courses/comsys/comsys-book.pdf#page=126)

ก่อนอื่นเราโหลดไลบรารีก่อน

In [ ]:
%%capture
!rm -rf comp-sys-public-lib
!wget -q -O - https://ecourse.cpe.ku.ac.th/courses/comsys/lib/ch06.tgz | tar zxf -
!cp comp-sys-public-lib/examples-ch06/*.asm .
!pip install -e comp-sys-public-lib
!pip install -e comp-sys-public-lib/modules/component-builder
import site; site.main()
from course_ch06_init import *

คำสั่ง `cp` ที่เพิ่มขึ้นมานั้นจะคัดลอกโค้ดภาษาแอสเซมบลี้ตัวอย่างมาไว้ในไดเรคตอรีทำงาน นิสิตสามารถนำมาทดสอบโปรแกรมแอสเซมเบลอร์ของตนเองได้  เปิดดูรายการไฟล์โดยกดไอคอนโฟลเดอร์ในแถบเครื่องมือด้านซ้ายมือของ Colab และดับเบิลคลิกเพื่อเปิดดูโค้ดในไฟล์

## สรุปชุดคำสั่งภาษาเครื่อง

เพื่อความสะดวกในการพัฒนาโปรแกรมแอสเซมเบลอร์ ขอนำเอารูปแบบคำสั่งภาษาเครื่องของ Hack Computer มาวางไว้ตรงนี้

### A-Instruction (Address Instruction)
มีรูปแบบเป็น

![a-instr-format.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/a-instr-format.png)

โดย `vvvvvvvvvvvvvvv` แทนจำนวนเต็ม 15 บิตเพื่อโหลดลงรีจีสเตอร์ `A`

### C-Instruction (Compute Instruction)
มีรูปแบบเป็น

![c-instr-format.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-format.png)

โดย
* `acccccc` หมายถึง compute field ใช้ควบคุมอินพุทของ ALU
* `ddd` หมายถึง destination field ใช้กำหนดที่เก็บเอาท์พุทของ ALU
* `jjj` หมายถึง jump field ใช้ร่วมกับขาเอาท์พุท (`zr`, `ng`) ของ ALU เพื่อกำหนดค่าให้กับรีจีสเตอร์ `PC`

#### ตาราง Compute Field (`acccccc`)

![c-instr-compute-field.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-compute-field.png?v=2)


#### ตาราง Destination Field (`ddd`)

![c-instr-dest-field.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-dest-field.png?v=2)


#### ตาราง Jump Field (`jjj`)

![c-instr-jump-field.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-jump-field.png?v=2)

## ข้อกำหนดของไฟล์อินพุทภาษา Hack Assembly

(คัดลอกจากหัวข้อ 6.2 ของตำราเรียน)

* **บรรทัดคำสั่ง** อนุญาตเพียง 1 คำสั่งต่อหนึ่งบรรทัด แต่ละบรรทัดเป็นไปได้ 2 รูปแบบคือ
    * **คำสั่ง (instruction)** ได้แก่บรรทัดที่เขียนคำสั่งในรูป A-instruction หรือ C-instruction
    * **ป้ายกระโดด (jump label)** เป็นคำสั่งที่เขียนในรูป `(Symbol)` ไม่นับเป็นคำสั่งจริงเพราะไม่ถูกแปลเป็นโค้ดภาษาเครื่อง แต่ใช้สำหรับแปะป้ายชื่อกำกับตำแหน่งหน่วยความจำให้กับคำสั่งจริงคำสั่งแรกที่พบถัดจากป้ายนี้ ป้ายกระโดดแต่ละชื่อ<u>ประกาศได้เพียงครั้งเดียว</u>เท่านั้นในโปรแกรมหนึ่ง ๆ
* **ช่องว่าง (whitespace)** อักขระเว้นวรรค (space) แท็บ (tab) และบรรทัดว่าง ไม่มีผลต่อโปรแกรมและไม่ต้องตีความใด ๆ
* **คอมเม้นต์ (comment)** คือข้อความที่นับตั้งแต่อักขระ `//` จนกระทั่งสุดบรรทัด ไม่ส่งผลกระทบใด ๆ กับโปรแกรม
* **ค่าคงที่ (constant)** ใช้กับ A-instruction ในรูป `@ddddd` โดย `ddddd` เป็น<u>จำนวนเต็มฐานสิบที่เป็นศูนย์หรือค่าบวกเท่านั้น</u> ไม่รองรับค่าติดลบ
* **ซิมโบล (symbol)** คือชื่อที่นำไปใช้งานในการนิยามป้ายกระโดดในรูป `(LABEL) ` หรือตัวแปรที่ใช้กับ A-instruction ในรูป `@variable`
    * ซิมโบลที่กำหนดไว้แล้ว (pre-defined symbol) เช่น `R0`, `R1`, ..., `R15`, `SCREEN` ฯลฯ ดูรายการทั้งหมดและค่าที่กำหนดให้ในตารางด้านล่าง
    * ซิมโบลที่กำหนดขึ้นเอง (user-defined symbol) ซึ่งได้แก่ตัวแปรและป้ายกระโดด ประกอบด้วยอักขระภาษาอังกฤษตัวใหญ่ (A-Z) อักษรตัวเล็ก (a-z) ตัวเลข (0-9) ขีดเส้นใต้ (`_`) จุด ​(`.`) สัญลักษณ์ดอลล่าร์ (`$`) และโคลอน (`:`) แต่ห้ามขึ้นต้นด้วยตัวเลข
* **อักษรตัวเล็ก/ตัวใหญ่** มีข้อบังคับและธรรมเนียมปฏิบัติดังนี้
    * คำสั่ง A-instruction และ C-instruction รวมถึงซิมโบลที่กำหนดไว้แล้ว (เช่น `R0`, `R1`, `SCREEN`) ต้องเขียนด้วย<u>อักษรตัวใหญ่</u>ทั้งหมด
    * ซิมโบลที่กำหนดเอง เขียนด้วยตัวเล็กหรือตัวใหญ่ก็ได้ ซึ่งจะถือว่าเป็นซิมโบลที่แตกต่างกัน แต่ตามธรรมเนียมปฏิบัติมักนิยามป้ายกระโดดด้วยอักษรตัวใหญ่ และนิยามตัวแปรด้วยอักษรเล็ก

<u>ตารางรายการซิมโบลที่กำหนดค่าให้แล้ว</u>

![predefined-symbols.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/predefined-symbols.png)

## การพัฒนา Hack Assembler

นิสิตสามารถออกแบบและเขียนโค้ดทั้งหมดเองได้ตามต้องการ โดยให้สามารถเรียกใช้งานผ่านฟังก์ชัน `assemble` ที่เขียนขึ้นเองตามข้อกำหนดของอินพุทและเอาท์พุทดังนี้

**ส่วนอินพุท**

ฟังก์ชัน `assemble` รับอินพุทภาษาแอสเซมบลี้ในรูปสตริง (เช่นเดียวกับที่ทำไปในสัปดาห์ที่ 4) ดังตัวอย่าง

```
assemble("""
    @R0
    M=1
(END)
    @END
    0;JMP
""")
```

หากต้องการอ่านอินพุทจากไฟล์จะต้องใช้ฟังก์ชัน `open()` ควบคู่กับเมท็อด `read()` เพื่ออ่านซอร์สโค้ดมาในรูปสตริงก่อน ดังตัวอย่าง

```
assemble(open('Add.asm').read())
```


**ส่วน output**

ฟังก์ชัน `assemble` สามารถเลือกให้ return เป็นลิสต์ของจำนวนเต็มที่แทนโค้ดคำสั่งภาษาเครื่อง หรือลิสต์ของสตริงที่แทนคำสั่งภาษาเครื่องในรูปแบบเลขฐานสองก็ได้ กล่าวคือ ผลลัพธ์แบบ

    [100, 200, 16, 50, 3]

กับ

    ['0000000001100100','0000000011001000','0000000000010000','0000000000110010','0000000000000011']

จะมีผลเหมือนกัน

### ขั้นตอนการพัฒนา

แนะนำขั้นตอนในการพัฒนาดังนี้ แต่นิสิตอาจข้ามไปทำขั้นตอนสุดท้ายได้เลย

* เขียนส่วน parser
  * อ่าน source code ทีละบรรทัดและตัด white space (hint: ใช้เมท็อดของสตริง เช่น `strip`, `split`, `replace` และอื่น ๆ)
  * แยกแยะว่าแต่ละบรรทัดเป็นป้ายกระโดด A-instruction หรือ C-instruction
  * ในบรรทัดที่เป็น C-instruction ตีความคำสั่งออกเป็นสามส่วนคือ `dest` `comp` และ `jump`
* เขียนแอสเซมเบลอร์แบบที่ไม่ต้องใช้ซิมโบล นั่นคือไม่มีการใช้ป้ายกระโดดหรือตัวแปร
  * เขียนส่วนสร้างรหัสภาษาเครื่องของ A-instructions (แบบที่ระบุค่าคงที่โดยตรง)
  * เขียนส่วนสร้างรหัสภาษาเครื่องของ C-instructions
  * ทดสอบกับซอร์สโค้ดที่ลงท้ายด้วย `L` เช่น `MaxL.asm` หรือ `RectL.asm`
* ปรับปรุงแอสเซมเบลอร์ให้ทำงานเป็น 2 pass
  * Pass 1: อ่านซอร์สโค้ดและสร้าง symbol table จากป้ายกระโดด (แนะนำให้ใช้ `dict` จะสะดวกมาก ๆ)
  * Pass 2: ปรับปรุงส่วนสร้างรหัสภาษาเครื่องของ A-instructions ให้สอบถามค่าจาก symbol table หรือเพิ่มรายการใหม่หากไม่พบค่า ซึ่งแสดงว่าเป็นตัวแปร

# โปรแกรมของคุณ

เขียนโปรแกรมลงในเซลด้านล่าง  หรืออาจจะทำในเครื่องคุณเองแล้วค่อยนำโค้ดมาทดลองที่ใน notebook นี้ก็ได้ เมื่อรันเซลด้านล่างแล้วจะต้องสามารถเรียกใช้ฟังก์ชัน `assemble` ได้ตามอินพุทและเอาท์พุทที่กำหนดข้างต้น หากต้องการสามารถประกาศตัวแปรเอาไว้ด้านนอกฟังก์ชัน `assemble` ได้

<font color="red">ข้อควรระวัง: ฟังก์ชัน `assemble` ต้องสามารถเรียกใช้งานได้มากกว่า 1 ครั้งโดยให้ผลลัพธ์ที่ถูกต้องเสมอ ดังนั้นตัวแปรที่ประกาศเอาไว้ด้านนอกฟังก์ชันควรเก็บค่าที่ฟังก์ชันอ่านไปใช้เพียงอย่างเดียว ไม่ควรให้ฟังก์ชันเขียนข้อมูลลงไปในตัวแปรได้ หรือหากจำเป็นต้องเขียนควรมีการตั้งค่าเริ่มต้นให้กับตัวแปรเหล่านั้นทุกครั้งที่ฟังก์ชัน `assemble` ถูกเรียกใช้งาน</font>

In [ ]:
def assemble(inp) :


  out = inp.splitlines() #Delete blank
  out = [i.strip() for i in out if i.strip() != '']


  for i in range(len(out)): #Dict @Address

    if '//' in out[i] :
      L,R = out[i].split('//')
      out[i] = L.strip()
  out = [i for i in out if i != '']
  return out

test = """//No
//yes
@0
"""

print(assemble(test))

['@0']


In [ ]:

# DO NOT ERASE THIS CELL - to be graded

# Put your complete assembler code here

def assemble(inp) :

  symbols = {
      "@SP":0,
      "@LCL":1,
      "@ARG":2,
      "@THIS":3,
      "@THAT":4,
      "@SCREEN":16384,
      "@KBD":24576
  }
  cnt = 16 #starter Address

  for i in range(16):
      symbols[f"@R{i}"] = i
    #Symbols

  c_dict = {
    "":  "000",
    "0":   "0101010",
    "1":   "0111111",
    "-1":  "0111010",
    "D":   "0001100",
    "A":   "0110000",
    "M":   "1110000",
    "!D":  "0001101",
    "!A":  "0110001",
    "!M":  "1110001",
    "-D":  "0001111",
    "-A":  "0110011",
    "-M":  "1110011",
    "D+1": "0011111",
    "A+1": "0110111",
    "M+1": "1110111",
    "D-1": "0001110",
    "A-1": "0110010",
    "M-1": "1110010",
    "D+A": "0000010",
    "D+M": "1000010",
    "D-A": "0010011",
    "D-M": "1010011",
    "A-D": "0000111",
    "M-D": "1000111",
    "D&A": "0000000",
    "D&M": "1000000",
    "D|A": "0010101",
    "D|M": "1010101",
    "JGT": "001",
    "JEQ": "010",
    "JGE": "011",
    "JLT": "100",
    "JNE": "101",
    "JLE": "110",
    "JMP": "111"
  }
  #C-Inst

  d_dict = {
      "":  "000",
      "M":   "001",
      "D":   "010",
      "MD":  "011",
      "A":   "100",
      "AM":  "101",
      "AD":  "110",
      "AMD": "111"
  } #Destination

  out = inp.splitlines()
  out = [i.strip() for i in out if i.strip() != ''] #Clear Space

  pc = 0 #PC
#Clear Comment
  for i in range(len(out)):
    if '//' in out[i] :
      L,R = out[i].split('//')
      out[i] = L.strip()
#Clear Blank
  out = [i for i in out if i != '']
#Set Jump point
  for i in range(len(out)):
    if '(' in out[i] :
      L,R = out[i].split(')')
      symbols['@'+L[1:]] = pc

      pc -= 1
      out[i] = ""
    pc += 1

  out = [i for i in out if i != '']

#Set Address For Symbol

  for i in range(len(out)):
    key = out[i]
    y = True
    if key in symbols :
      y = False
    try :
       x = isinstance(int(out[i][1]),int)

    except :
      x = False
    if out[i][0] == '@' and x : #@ Address
      symbols[out[i]] = int(out[i][1:])
    elif out[i][0] == '@' and y :  #@ Symbol
      symbols[out[i]] = cnt
      cnt+=1

#Clear Blank
  out = [i for i in out if i != '']
#Change Symbol to value

  for i in range(len(out)): #Symbol -> value
    try :
      out[i] = symbols[out[i]] #If A inst
    except : #For C inst
      s = out[i]
      Jump,Destination = "" ,""
      if "=" in s:
        Destination, Nimo = s.split("=")
      elif ";" in s:
        Nimo, Jump = s.split(";")

      Destination = d_dict[Destination]
      Nimo = c_dict[Nimo]
      Jump = c_dict[Jump]

      out[i] = '111'+Nimo+Destination+Jump
      out[i] = int(out[i],2)

  return out

In [ ]:

ok = course_assemble(test)
test == ok
def diff_by_index(a, b):
    MISSING = object()
    diffs = []
    maxlen = max(len(a), len(b))
    for i in range(maxlen):
        va = a[i] if i < len(a) else MISSING
        vb = b[i] if i < len(b) else MISSING
        if va != vb:
            diffs.append((i, None if va is MISSING else va,
                             None if vb is MISSING else vb))
    return diffs

# ตัวอย่าง
A = [1, 2, 2, 3, 5]
B = [1, 2, 4, 3]
print(diff_by_index(test, ok))
# [(2, 2, 4), (4, 5, None)]
# ความหมาย: index 2 ต่างกัน (A=2,B=4), index 4 มีใน A แต่ไม่มีใน B


[(0, '/', 2), (1, '/', 60432), (2, ' ', 3), (3, 'T', 57488), (4, 'h', 0), (5, 'i', 58120), (6, 's', None), (7, ' ', None), (8, 'f', None), (9, 'i', None), (10, 'l', None), (11, 'e', None), (12, ' ', None), (13, 'i', None), (14, 's', None), (15, ' ', None), (16, 'p', None), (17, 'a', None), (18, 'r', None), (19, 't', None), (20, ' ', None), (21, 'o', None), (22, 'f', None), (23, ' ', None), (24, 'w', None), (25, 'w', None), (26, 'w', None), (27, '.', None), (28, 'n', None), (29, 'a', None), (30, 'n', None), (31, 'd', None), (32, '2', None), (33, 't', None), (34, 'e', None), (35, 't', None), (36, 'r', None), (37, 'i', None), (38, 's', None), (39, '.', None), (40, 'o', None), (41, 'r', None), (42, 'g', None), (43, '\n', None), (44, '/', None), (45, '/', None), (46, ' ', None), (47, 'a', None), (48, 'n', None), (49, 'd', None), (50, ' ', None), (51, 't', None), (52, 'h', None), (53, 'e', None), (54, ' ', None), (55, 'b', None), (56, 'o', None), (57, 'o', None), (58, 'k', None), (59, ' ', N

## การทดสอบ

### ทดสอบโดยการเปรียบเทียบผลลัพธ์

เมื่อเริ่มต้นเขียนบางส่วน นิสิตอาจจะต้องการตรวจสอบการแปลงโปรแกรมสั้น ๆ  นิสิตสามารถเทียบผลลัพธ์ที่ได้กับฟังก์ชัน `course_assemble` ที่เป็นโปรแกรมแอสเซมเบลอร์ที่รายวิชาเตรียมไว้ให้ หากต้องการทดสอบเทียบการแปลงกับตัวอย่างไฟล์โปรแกรม ให้เรียกคำสั่งในลักษณะนี้

    course_assemble(open("Add.asm").read())

ผลลัพธ์ที่ได้จะเป็นลิสต์ของจำนวนเต็ม  ถ้าต้องการผลลัพธ์เป็นแบบสตริงเลขฐานสองให้ใช้คำสั่งดังนี้

    [f'{inst:016b}' for inst in course_assemble(open("Add.asm").read())]

แล้วเทียบกับผลลัพธ์ที่นิสิตสร้างได้


In [ ]:
# พื้นที่ทดสอบ
out1 = assemble(open("Pong.asm").read())
out2 = course_assemble(open("Pong.asm").read())
out1 == out2

True

### ทดสอบด้วยซิมูเลชัน

นิสิตสามารถทดลองรันโปรแกรมบน Hack CPU โดยเรียก `simulate` เช่นที่เคยทำมา หลาย ๆ โปรแกรมทดสอบต้องอ่านค่าจาก `R0` หรือ `R1` (ดูรายละเอียดในคอมเม้นต์ที่ต้นไฟล์ตัวอย่างต่าง ๆ) ดังนั้นอย่าลืมตั้งค่าก่อนที่จะรันโปรแกรม

อย่างไรก็ตาม ถ้าต้องการรัน `Pong` ให้เรียกโดยเพิ่มอ็อปชัน `super_fast=True` เข้าไปด้วย เพื่อให้ซิมูเลชันทำงานได้เร็วพอ เช่น

    simulate(pong_instructions, super_fast=True)

ถ้ายังเร็วไม่พอ สามารถเติม option `fastest_steps=10003` ได้ด้วย โดยลองเปลี่ยนเลข `10003` เป็นเลขต่าง ๆ

In [ ]:
# พื้นที่ทดสอบ
simulate(assemble(open("Pong.asm").read()), super_fast=True)